In [99]:
df = pd.read_csv('./merge_data.csv', encoding='cp949')
df = df.sample(frac=1, random_state=7777).reset_index(drop=True)

In [100]:
df_train = df[df['round'] != 2]
df_valid = df[df['round'] == 2]

In [92]:
df.columns

Index(['Unnamed: 0', 'timestamp_rounded', 'RearHeatPower266',
       'DI_accelPedalPos', 'Unnamed: 4', 'DI_regenLight', 'DI_vehicleSpeed',
       'DI_regenLight.1', 'DIR_torqueActual', 'RearTorque1D8',
       'SystemHeatPowerMax268', 'SmoothBattCurrent132', 'DIR_torqueCommand',
       'DI_regenLight.2', 'SteeringSpeed129', 'DI_vehicleSpeed.1',
       'RearHeatPower266_mean', 'RearHeatPower266_std',
       'RearHeatPower266_median', 'DI_accelPedalPos_mean',
       'DI_accelPedalPos_std', 'DI_accelPedalPos_median', 'DI_regenLight_mean',
       'DI_regenLight_std', 'DI_regenLight_median', 'DI_vehicleSpeed_mean',
       'DI_vehicleSpeed_std', 'DI_vehicleSpeed_median', 'DI_regenLight.1_mean',
       'DI_regenLight.1_std', 'DI_regenLight.1_median',
       'DIR_torqueActual_mean', 'DIR_torqueActual_std',
       'DIR_torqueActual_median', 'RearTorque1D8_mean', 'RearTorque1D8_std',
       'RearTorque1D8_median', 'SystemHeatPowerMax268_mean',
       'SystemHeatPowerMax268_std', 'SystemHeatPowerM

In [101]:
# 이후의 데이터 처리 코드
train_data = df_train.drop(['Unnamed: 4','Unnamed: 0','round', 'timestamp_rounded'], axis=1)
valid_data = df_valid.drop(['Unnamed: 4','Unnamed: 0','round', 'timestamp_rounded'], axis=1)

In [102]:
train_data.reset_index(inplace=True, drop=True)
valid_data.reset_index(inplace=True, drop=True)

In [24]:
# X_train = train_data.drop('label', axis=1)
# y_train = train_data['label']

# X_valid = valid_data.drop('label', axis=1)
# y_valid = valid_data['label']

In [103]:
# 결측치 drop
X_train = X_train.fillna(method='ffill')
y_train = y_train[X_train.index]

X_valid = X_valid.fillna(method='ffill')
y_valid = y_valid[X_valid.index]

In [104]:
X_train = train_data.drop('label', axis=1)
y_train = train_data['label']

X_valid = valid_data.drop('label', axis=1)
y_valid = valid_data['label']

In [105]:
# 결측치 확인
print(X_train.isnull().sum().sum())  # 모든 결측치의 수를 출력

# SimpleImputer를 사용하여 결측치를 평균값으로 채움
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)

# 결측치 처리 후 데이터 검증
print(pd.DataFrame(X_train_imputed).isnull().sum().sum())  # 모든 결측치의 수를 출력

# X_valid에 대한 결측치 처리
X_valid_imputed = imputer.transform(X_valid)  # 이미 학습된 imputer 사용

# 결측치 처리 후 데이터 검증
print(pd.DataFrame(X_valid_imputed).isnull().sum().sum())  # 모든 결측치의 수를 출력

# 모델 학습
mlp_model = MLPClassifier(random_state=42, max_iter=500)  # max_iter를 증가
mlp_model.fit(X_train_imputed, y_train)

# 모델 예측
y_pred_mlp = mlp_model.predict(X_valid_imputed)

joblib.dump(mlp_model, 'mlp_model_round_2.pkl')

36
0
0


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


['mlp_model_round_2.pkl']

In [106]:
import joblib
import numpy as np
import pandas as pd

from sklearn.impute import SimpleImputer

# SimpleImputer 인스턴스 생성 (이미 X_train에 적용된 방식과 동일해야 함)
imputer = SimpleImputer(strategy='mean')

# X_valid에 결측치 처리
X_valid_imputed = imputer.fit_transform(X_valid)

# 모델 불러오기 및 예측 확률 계산
rf_model = joblib.load('mlp_model_round_2.pkl')
y_pred_prob = rf_model.predict_proba(X_valid_imputed)

# 후속 코드...


# # 저장된 모델 불러오기
# rf_model = joblib.load('mlp_model_round_1.pkl')
# # 예측 확률 계산
# y_pred_prob = rf_model.predict_proba(X_valid)

# 레이블에 따라 확률을 그룹화
grouped_prob = {label: [] for label in np.unique(y_valid)}
for prob, label in zip(y_pred_prob, y_valid):
    grouped_prob[label].append(prob)

# 각 레이블에 대한 평균 확률 계산
average_probs = {label: np.mean(probs, axis=0) for label, probs in grouped_prob.items()}

# 평균 확률을 DataFrame으로 변환
average_probs_df = pd.DataFrame(average_probs).transpose()
average_probs_df.columns = [f'{i}' for i in range(y_pred_prob.shape[1])]
average_probs_df['label'] = average_probs_df.index

# 'label' 열을 맨 앞으로 이동
column_order = ['label'] + [col for col in average_probs_df if col != 'label']
average_probs_df = average_probs_df[column_order]

# DataFrame 생성 및 CSV로 저장
average_probs_df.to_csv('./2_avg_prob.csv', index=False, float_format='%.3f')